In [4]:
import os
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import os, csv, hashlib
from pathlib import Path
from collections import defaultdict, Counter
from PIL import Image, UnidentifiedImageError, ImageFile

import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import shutil
from tqdm import tqdm
import random
from pathlib import Path

import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import random
from pathlib import Path
from pathlib import Path

import os
import random
import cv2
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

import os, json, csv, random
from collections import Counter, defaultdict
from collections import Counter, defaultdict
import pandas as pd
import matplotlib.pyplot as plt

import os, csv
from pathlib import Path
from sklearn.model_selection import train_test_split
from pathlib import Path
import pandas as pd
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
import matplotlib.pyplot as plt

In [5]:
dataset = "Tea Leaf Disease Dataset"

In [8]:
# pipeline.py (complete pipeline with all 6 preprocessing steps)
import os, csv, hashlib, shutil, random
from pathlib import Path
from collections import defaultdict, Counter
from typing import List, Tuple
from PIL import Image, UnidentifiedImageError
import numpy as np
import torch
import torchvision.transforms as T
import pandas as pd

# ---------------- PATH RESOLVER ----------------
def resolve_raw_root() -> Path:
    """Find '../Data/Raw/Tea Leaf Disease Dataset' (or close variants)."""
    name = "Tea Leaf Disease Dataset"
    cwd = Path.cwd()
    candidates = [
        cwd / "Data" / "Raw" / name,
        cwd / ".." / "Data" / "Raw" / name,
        cwd / ".." / ".." / "Data" / "Raw" / name,
        cwd / name,
        cwd / ".." / name,
    ]
    for p in candidates:
        if p.is_dir():
            print(f"Using RAW_ROOT: {p.resolve()}")
            return p.resolve()
    tried = "\n - ".join(str(p.resolve()) for p in candidates)
    raise FileNotFoundError(f"Couldn't find raw dataset folder. Tried:\n{tried}")

RAW_ROOT = resolve_raw_root()

# ---------------- CONFIG ----------------
RESULTS_DIR     = Path("../results")
PREP_IMG_DIR    = RESULTS_DIR / "Preprocessed_Images"
SPLITS_DIR      = Path("../Data/Splits")
OVERSAMPLED_CSV = RESULTS_DIR / "Splits" / "train_oversampled.csv"
LOG_CSV         = RESULTS_DIR / "clean_log.csv"

PREP_IMG_DIR.mkdir(parents=True, exist_ok=True)
(RESULTS_DIR / "Splits").mkdir(parents=True, exist_ok=True)

# Cleaning
SCAN_EXTS        = (".jpg", ".jpeg", ".png", ".bmp", ".webp", ".tif", ".tiff")
DISALLOWED_EXTS  = (".bmp", ".webp", ".tif", ".tiff", ".jpeg")
REQUIRE_RGB      = True
MIN_SIDE         = 64
MAX_ASPECT       = 3.0
CHECK_MD5_DUPES  = True
CHECK_PHASH_NEAR = True

# Resizing
TARGET_SIZE      = (224, 224)
SAVE_EXT         = ".jpg"

# Normalization
NORM_MEAN = [0.485, 0.456, 0.406]
NORM_STD  = [0.229, 0.224, 0.225]
NORM_DIR = RESULTS_DIR / "Normalized_Arrays"
NORM_PREVIEW_DIR = RESULTS_DIR / "Normalized_Previews"

# Augmentation
N_AUG = 2
AUG_TF = T.Compose([
    T.RandomResizedCrop((TARGET_SIZE[1], TARGET_SIZE[0]), scale=(0.8, 1.0), ratio=(3/4, 4/3)),
    T.RandomHorizontalFlip(p=0.5),
    T.RandomVerticalFlip(p=0.1),
    T.RandomRotation(degrees=15),
    T.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.05),
    T.RandomPerspective(distortion_scale=0.25, p=0.2),
])
AUG_IMG_DIR = RESULTS_DIR / "Augmented_Images"

# Splits
TEST_SIZE        = 0.20
VAL_SIZE         = 0.10
RANDOM_STATE     = 42

# Oversampling
OVERSAMPLE_TO_MAX = True

# ---------------- HELPERS ----------------
def list_class_dirs(root: Path) -> List[Path]:
    return sorted([d for d in root.iterdir() if d.is_dir() and not d.name.startswith("_")])

def file_md5(path: Path, chunk=8192) -> str:
    m = hashlib.md5()
    with path.open("rb") as f:
        for blk in iter(lambda: f.read(chunk), b""):
            m.update(blk)
    return m.hexdigest()

def phash_hex(path: Path, hash_size=8) -> str:
    with Image.open(path) as im:
        im = im.convert("L").resize((hash_size, hash_size), Image.LANCZOS)
        px = np.array(im, dtype=np.float32)
        avg = px.mean()
        bits = (px > avg).astype(np.uint8).flatten()
        return f"{int(''.join(map(str,bits)), 2):016x}"

def safe_save(img: Image.Image, dst: Path):
    dst.parent.mkdir(parents=True, exist_ok=True)
    base, ext = os.path.splitext(dst)
    k, final = 1, dst
    while Path(final).exists():
        final = Path(f"{base}__dup{k}{ext}")
        k += 1
    img.save(final, quality=95)

def resize_img(p: Path, size=(224,224)) -> Image.Image:
    with Image.open(p) as im:
        im = im.convert("RGB")
        return im.resize(size, Image.BILINEAR)

# ---------------- STEP 1: CLEAN ----------------
def scan_and_clean(root: Path):
    issues, survivors_by_class = [], defaultdict(list)
    before_counts, after_counts = {}, {}

    for cls_dir in list_class_dirs(root):
        cls = cls_dir.name
        files = [f for f in cls_dir.iterdir() if f.suffix.lower() in SCAN_EXTS]
        before_counts[cls] = len(files)
        valid = 0

        for f in files:
            ext = f.suffix.lower()
            if ext in DISALLOWED_EXTS:
                issues.append([cls, str(f), f"Disallowed extension: {ext}"])
                continue
            if f.stat().st_size == 0:
                issues.append([cls, str(f), "Zero-byte file"])
                continue
            try:
                with Image.open(f) as im: im.verify()
                with Image.open(f) as im:
                    im.load()
                    mode, (w,h) = im.mode, im.size
            except Exception as e:
                issues.append([cls, str(f), f"Unreadable: {e}"])
                continue
            if REQUIRE_RGB and mode != "RGB":
                issues.append([cls, str(f), f"Invalid mode: {mode}"])
                continue
            if min(w,h) < MIN_SIDE or max(w,h)/max(1,min(w,h)) > MAX_ASPECT:
                issues.append([cls, str(f), f"Invalid size: {w}x{h}"])
                continue
            survivors_by_class[cls].append(f); valid += 1
        after_counts[cls] = valid

    # Duplicates
    if CHECK_MD5_DUPES or CHECK_PHASH_NEAR:
        for cls, paths in survivors_by_class.items():
            keep = paths
            if CHECK_MD5_DUPES:
                md5_map = defaultdict(list)
                for p in paths: md5_map[file_md5(p)].append(p)
                keep = [group[0] for group in md5_map.values()]
            if CHECK_PHASH_NEAR:
                phash_map = defaultdict(list)
                for p in keep: phash_map[phash_hex(p)].append(p)
                keep = [group[0] for group in phash_map.values()]
            survivors_by_class[cls] = keep
            after_counts[cls] = len(keep)

    with LOG_CSV.open("w", newline="", encoding="utf-8") as f:
        w = csv.writer(f); w.writerow(["class","path","reason"]); w.writerows(issues)

    return survivors_by_class, before_counts, after_counts, issues

# ---------------- STEP 2: RESIZE ----------------
def write_preprocessed_images(survivors_by_class, out_dir: Path, size=(224,224)):
    out_dir.mkdir(parents=True, exist_ok=True)
    count = 0
    for cls, files in survivors_by_class.items():
        dst_cls = out_dir / cls
        dst_cls.mkdir(parents=True, exist_ok=True)
        for f in files:
            try:
                imr = resize_img(f, size=size)
                dst = dst_cls / (f.stem + SAVE_EXT)
                safe_save(imr, dst); count += 1
            except Exception: pass
    return count

# ---------------- STEP 2.5: NORMALIZATION ----------------
def offline_normalize_all(prep_root: Path, npy_dir: Path, preview_dir: Path, limit_preview_per_class=10):
    npy_dir.mkdir(parents=True, exist_ok=True)
    preview_dir.mkdir(parents=True, exist_ok=True)
    saved_count, preview_count = 0, defaultdict(int)

    def to_tensor(img): return T.ToTensor()(img)
    norm = T.Normalize(mean=NORM_MEAN, std=NORM_STD)

    for cls_dir in list_class_dirs(prep_root):
        cls = cls_dir.name
        out_cls_npy, out_cls_prev = npy_dir/cls, preview_dir/cls
        out_cls_npy.mkdir(parents=True, exist_ok=True)
        out_cls_prev.mkdir(parents=True, exist_ok=True)
        for f in cls_dir.glob("*.jpg"):
            try:
                with Image.open(f) as im:
                    im = im.convert("RGB")
                    x = to_tensor(im); x_norm = norm(x)
                    np.save(out_cls_npy / (f.stem + ".npy"), x_norm.numpy().astype('float32'))
                    saved_count += 1
                    if preview_count[cls] < limit_preview_per_class:
                        mean = torch.tensor(NORM_MEAN).view(3,1,1)
                        std  = torch.tensor(NORM_STD).view(3,1,1)
                        x_prev = (x_norm*std + mean).clamp(0,1)
                        Image.fromarray((x_prev.permute(1,2,0).numpy()*255).astype('uint8')).save(
                            out_cls_prev / (f.stem+"_denorm.jpg"), quality=95)
                        preview_count[cls]+=1
            except Exception: pass
    print(f"Normalized .npy saved: {saved_count}")

# ---------------- STEP 3: SPLITS ----------------
def list_images_with_labels(root: Path) -> Tuple[List[str], List[str]]:
    paths, labels = [], []
    for cls_dir in list_class_dirs(root):
        cls = cls_dir.name
        for f in cls_dir.glob("*.jpg"):
            rel = f.relative_to(root); paths.append(str(rel)); labels.append(cls)
    return paths, labels

def ensure_splits(prep_root: Path, splits_dir: Path, test=0.20, val=0.10, seed=42):
    tcsv, vcsv, scsv = splits_dir/"train_index.csv", splits_dir/"val_index.csv", splits_dir/"test_index.csv"
    if tcsv.exists() and vcsv.exists() and scsv.exists(): return
    from sklearn.model_selection import train_test_split
    X, y = list_images_with_labels(prep_root)
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=test+val, stratify=y, random_state=seed)
    rel_val = val/(test+val)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=1-rel_val, stratify=y_temp, random_state=seed)
    splits_dir.mkdir(parents=True, exist_ok=True)
    for name, Xs, Ys in [("train_index.csv",X_train,y_train),("val_index.csv",X_val,y_val),("test_index.csv",X_test,y_test)]:
        with (splits_dir/name).open("w", newline="") as f: csv.writer(f).writerows([["path","label"],*zip(Xs,Ys)])

# ---------------- STEP 3.5: AUGMENTATION ----------------
def augment_training_split(prep_root: Path, splits_dir: Path, out_img_dir: Path, n_aug=N_AUG):
    train_csv = splits_dir / "train_index.csv"; assert train_csv.exists()
    rows = pd.read_csv(train_csv).to_records(index=False)
    aug_rows, total_aug = [], 0
    out_img_dir.mkdir(parents=True, exist_ok=True)
    for rel_path, label in rows:
        try:
            im = Image.open(prep_root/rel_path).convert("RGB")
            dst_cls = out_img_dir/label; dst_cls.mkdir(parents=True, exist_ok=True)
            for k in range(n_aug):
                aug_img = AUG_TF(im); aug_name = f"{Path(rel_path).stem}_aug{k}{SAVE_EXT}"
                aug_path = dst_cls/aug_name; aug_img.save(aug_path, quality=95)
                aug_rows.append((str(aug_path.relative_to(out_img_dir)), label)); total_aug+=1
        except Exception: pass
    combined_csv = splits_dir / "train_index_augmented.csv"
    with combined_csv.open("w", newline="") as f:
        w=csv.writer(f); w.writerow(["root","path","label"])
        for rel_path,label in rows: w.writerow([str(prep_root.resolve()), rel_path, label])
        for rel_aug,label in aug_rows: w.writerow([str(out_img_dir.resolve()), rel_aug, label])
    print(f"Augmented images: {total_aug}")

# ---------------- STEP 4: OVERSAMPLING ----------------
def write_oversampled_csv(train_csv: Path, out_csv: Path):
    df = pd.read_csv(train_csv)
    by_cls = df.groupby("label")["path"].apply(list).to_dict()
    max_count = max(len(v) for v in by_cls.values())
    rows = []
    for cls, paths in by_cls.items():
        take = (paths*((max_count+len(paths)-1)//len(paths)))[:max_count]
        rows.extend([(p,cls) for p in take])
    random.seed(RANDOM_STATE); random.shuffle(rows)
    out_csv.parent.mkdir(parents=True, exist_ok=True)
    with out_csv.open("w", newline="") as f: csv.writer(f).writerows([["path","label"],*rows])
    print("Oversampled train CSV:", out_csv.resolve())

# ---------------- MAIN ----------------
if __name__=="__main__":
    survivors, before, after, issues = scan_and_clean(RAW_ROOT)
    print("CLEAN SUMMARY:"); [print(c,before[c],after[c]) for c in sorted(before)]
    count = write_preprocessed_images(survivors, PREP_IMG_DIR, size=TARGET_SIZE)
    print(f"Wrote {count} resized images -> {PREP_IMG_DIR}")
    offline_normalize_all(PREP_IMG_DIR, NORM_DIR, NORM_PREVIEW_DIR)
    ensure_splits(PREP_IMG_DIR, SPLITS_DIR, test=TEST_SIZE, val=VAL_SIZE, seed=RANDOM_STATE)
    augment_training_split(PREP_IMG_DIR, SPLITS_DIR, AUG_IMG_DIR, n_aug=N_AUG)
    train_csv = SPLITS_DIR/"train_index.csv"
    if train_csv.exists(): write_oversampled_csv(train_csv, OVERSAMPLED_CSV)
    print("\n Pipeline complete.")

Using RAW_ROOT: /Users/tharukakumarasiri/Desktop/AI_ML_Project/Data/Raw/Tea Leaf Disease Dataset
CLEAN SUMMARY:
algal_spot 5497 5135
brown_blight 4908 4810
gray_blight 5537 5121
healthy 5492 4990
helopeltis 5482 5254
red_spot 5505 5266
Wrote 30576 resized images -> ../results/Preprocessed_Images
Normalized .npy saved: 61152
Augmented images: 42806
Oversampled train CSV: /Users/tharukakumarasiri/Desktop/results/Splits/train_oversampled.csv

 Pipeline complete.


In [9]:
from pathlib import Path

# What does the pipeline think "results" is?
NOTEBOOK_CWD = Path.cwd()
PROJECT_ROOT = NOTEBOOK_CWD.parent       # because your notebooks live in .../AI_ML_Project/Notebooks
RESULTS_DIR  = PROJECT_ROOT / "results"
PREP_IMG_DIR = RESULTS_DIR / "Preprocessed_Images"
AUG_IMG_DIR  = RESULTS_DIR / "Augmented_Images"
NORM_DIR     = RESULTS_DIR / "Normalized"

print("Notebook CWD        :", NOTEBOOK_CWD)
print("Project root (expected):", PROJECT_ROOT)
print("RESULTS_DIR (expected):", RESULTS_DIR.resolve())
print("Preprocessed dir     :", PREP_IMG_DIR.resolve())
print("Exists?              :", PREP_IMG_DIR.exists())

# How many files do we actually have there?
jpgs = list(PREP_IMG_DIR.rglob("*.jpg")) if PREP_IMG_DIR.exists() else []
print("Sample count in Preprocessed_Images:", len(jpgs))
print("First 5:", [p.relative_to(PROJECT_ROOT) for p in jpgs[:5]])

Notebook CWD        : /Users/tharukakumarasiri/Desktop/AI_ML_Project
Project root (expected): /Users/tharukakumarasiri/Desktop
RESULTS_DIR (expected): /Users/tharukakumarasiri/Desktop/results
Preprocessed dir     : /Users/tharukakumarasiri/Desktop/results/Preprocessed_Images
Exists?              : True
Sample count in Preprocessed_Images: 61152
First 5: [PosixPath('results/Preprocessed_Images/healthy/healthy_0_7840__dup1.jpg'), PosixPath('results/Preprocessed_Images/healthy/healthy_0_2462__dup1.jpg'), PosixPath('results/Preprocessed_Images/healthy/healthy_0_7889__dup1.jpg'), PosixPath('results/Preprocessed_Images/healthy/healthy_0_2181__dup1.jpg'), PosixPath('results/Preprocessed_Images/healthy/healthy_0_9894__dup1.jpg')]


In [10]:
import os, csv, json, random, shutil
from pathlib import Path
from collections import defaultdict, Counter
from typing import Dict, List
import cv2
from PIL import Image
from tqdm import tqdm

# ---------- CONFIG ----------
PREP_IMG_DIR   = Path("../results/Preprocessed_Images")  # cleaned + resized 224x224 JPGs
AUG_IMG_DIR    = Path("../results/Augmented_Images")     # offline augs (if you created them)
SPLITS_DIR     = Path("../Data/Splits")                  # train/val/test CSVs here

FINAL_DIR      = Path("../results/Final_Preprocessed_Dataset")  # <-- ONE folder
INCLUDE_AUG    = True         # put all augmented images into train
ADD_COLOR_CONV = True         # add HSV/LAB/YUV converted copies (train only)
BALANCE_TRAIN  = True         # oversample (by copying) to max class size (train only)
SEED           = 42

# Normalization you’ll use during training (recorded in _meta/norm_stats.json)
NORM_MEAN = [0.485, 0.456, 0.406]
NORM_STD  = [0.229, 0.224, 0.225]

# ---------- UTILS ----------
def read_split(csv_path: Path) -> List[tuple]:
    rows = []
    with csv_path.open() as f:
        r = csv.DictReader(f)
        for row in r:
            rows.append((row["path"], row["label"]))
    return rows

def copy_with_unique(src: Path, dst: Path):
    dst.parent.mkdir(parents=True, exist_ok=True)
    base = dst.with_suffix('')
    ext = dst.suffix
    i, final = 1, dst
    while final.exists():
        final = Path(str(base) + f"__dup{i}" + ext)
        i += 1
    shutil.copy2(src, final)
    return final

def color_convert_variants(img_bgr):
    out = []
    # HSV
    hsv = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2HSV)
    hsv_bgr = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    out.append(("hsv", hsv_bgr))
    # LAB
    lab = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2LAB)
    lab_bgr = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)
    out.append(("lab", lab_bgr))
    # YUV
    yuv = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2YUV)
    yuv_bgr = cv2.cvtColor(yuv, cv2.COLOR_YUV2BGR)
    out.append(("yuv", yuv_bgr))
    return out

def gather_augmented_for_class(cls: str) -> List[Path]:
    """Return all augmented files for a class if AUG_IMG_DIR exists."""
    if not AUG_IMG_DIR.exists():
        return []
    cls_dir = AUG_IMG_DIR / cls
    if not cls_dir.exists():
        return []
    return [p for p in cls_dir.glob("*.jpg")]

def ensure_clean_dir(path: Path):
    if path.exists():
        shutil.rmtree(path)
    path.mkdir(parents=True, exist_ok=True)

# ---------- BUILD ----------
def build_final_dataset():
    random.seed(SEED)

    # 0) Read splits
    train_rows = read_split(SPLITS_DIR / "train_index.csv")
    val_rows   = read_split(SPLITS_DIR / "val_index.csv")
    test_rows  = read_split(SPLITS_DIR / "test_index.csv")

    ensure_clean_dir(FINAL_DIR)
    (FINAL_DIR / "train").mkdir()
    (FINAL_DIR / "val").mkdir()
    (FINAL_DIR / "test").mkdir()
    (FINAL_DIR / "_meta").mkdir()

    # 1) Copy base preprocessed images for each split
    def copy_split(rows, split_name):
        per_class_paths = defaultdict(list)
        for rel, cls in tqdm(rows, desc=f"Copying base {split_name}"):
            src = PREP_IMG_DIR / rel  # paths in CSV are relative to PREP_IMG_DIR
            if not src.exists():
                # fall back: some CSVs may store just filename; try class join
                src = PREP_IMG_DIR / cls / Path(rel).name
            if not src.exists():
                # skip silently if truly missing
                continue
            dst = FINAL_DIR / split_name / cls / src.name
            final = copy_with_unique(src, dst)
            per_class_paths[cls].append(final)
        return per_class_paths

    train_paths = copy_split(train_rows, "train")
    val_paths   = copy_split(val_rows,   "val")
    test_paths  = copy_split(test_rows,  "test")

    # 2) Add augmentations to TRAIN
    aug_added = 0
    if INCLUDE_AUG and AUG_IMG_DIR.exists():
        for cls in train_paths.keys():
            aug_files = gather_augmented_for_class(cls)
            for src in tqdm(aug_files, desc=f"Adding aug -> train/{cls}", leave=False):
                dst = FINAL_DIR / "train" / cls / src.name
                copy_with_unique(src, dst)
                aug_added += 1

    # 3) Add colour-conversion variants for TRAIN (optional)
    cc_added = 0
    if ADD_COLOR_CONV:
        for cls in train_paths.keys():
            cls_dir = FINAL_DIR / "train" / cls
            originals = [p for p in cls_dir.glob("*.jpg")]
            for src in tqdm(originals, desc=f"Color convert -> train/{cls}", leave=False):
                img = cv2.imread(str(src))
                if img is None:
                    continue
                for tag, var in color_convert_variants(img):
                    dst = src.with_name(src.stem + f"__{tag}" + src.suffix)
                    if not dst.exists():
                        cv2.imwrite(str(dst), var)
                        cc_added += 1

    # 4) Balance TRAIN by oversampling (duplicate copies) to max class size
    balanced_added = 0
    if BALANCE_TRAIN:
        counts = {}
        for cls in (FINAL_DIR / "train").iterdir():
            if cls.is_dir():
                counts[cls.name] = len(list(cls.glob("*.jpg")))
        if counts:
            target = max(counts.values())
            for cls, n in counts.items():
                if n >= target:
                    continue
                cls_dir = FINAL_DIR / "train" / cls
                imgs = list(cls_dir.glob("*.jpg"))
                need = target - n
                for i in range(need):
                    src = imgs[i % len(imgs)]
                    dst = cls_dir / (src.stem + f"__os{i}" + src.suffix)
                    shutil.copy2(src, dst)
                    balanced_added += 1

    # 5) Write meta info (normalization and counts)
    meta = {
        "normalization": {"mean": NORM_MEAN, "std": NORM_STD},
        "include_aug": INCLUDE_AUG,
        "add_color_converted": ADD_COLOR_CONV,
        "balanced_train": BALANCE_TRAIN,
    }
    with (FINAL_DIR / "_meta" / "norm_stats.json").open("w") as f:
        json.dump(meta, f, indent=2)

    # simple count report
    report_lines = []
    for split in ["train", "val", "test"]:
        split_dir = FINAL_DIR / split
        per_cls = {d.name: len(list((split_dir / d.name).glob("*.jpg")))
                   for d in split_dir.iterdir() if d.is_dir()}
        total = sum(per_cls.values())
        report_lines.append(f"{split}: {total} images\n" +
                            "\n".join([f"  - {k}: {v}" for k,v in sorted(per_cls.items())]))
    report_lines.append(f"\nAugmentations added: {aug_added}")
    report_lines.append(f"Color-converted added: {cc_added}")
    report_lines.append(f"Balanced copies added: {balanced_added}")

    (FINAL_DIR / "_meta" / "build_report.txt").write_text("\n\n".join(report_lines))
    print("\n=== FINAL DATASET BUILT ===")
    print("\n".join(report_lines))
    print("\nSaved to:", FINAL_DIR.resolve())

if __name__ == "__main__":
    build_final_dataset()

Copying base test: 100%|██████████| 6116/6116 [00:03<00:00, 1738.34it/s]



=== FINAL DATASET BUILT ===
train: 265392 images
  - algal_spot: 44232
  - brown_blight: 44232
  - gray_blight: 44232
  - healthy: 44232
  - helopeltis: 44232
  - red_spot: 44232
val: 3057 images
  - algal_spot: 514
  - brown_blight: 481
  - gray_blight: 512
  - healthy: 499
  - helopeltis: 525
  - red_spot: 526
test: 6116 images
  - algal_spot: 1027
  - brown_blight: 962
  - gray_blight: 1024
  - healthy: 998
  - helopeltis: 1051
  - red_spot: 1054

Augmentations added: 42806
Color-converted added: 192627
Balanced copies added: 8556

Saved to: /Users/tharukakumarasiri/Desktop/results/Final_Preprocessed_Dataset
